# DeepFake using first-order-motion-model

**Clone repository**

In [1]:
!git clone https://github.com/devasenan134/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 327, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 327 (delta 12), reused 32 (delta 10), pack-reused 293
Receiving objects: 100% (327/327), 77.69 MiB | 30.41 MiB/s, done.
Resolving deltas: 100% (163/163), done.


In [2]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


**Add folder https://drive.google.com/drive/folders/19CEDADQyeNayTNDp-eZ4EDDb0l3XhQmP to your google drive**

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from math import ceil
print("done import")

done import


**Utils**

In [24]:
root_path = "/content/gdrive/MyDrive/ColabNotebooks/ML/first-order-motion-model/data"

def img_vid_resize(source, cap, size_x, size_y):
    source_image = cv2.resize(source, (size_x, size_y))
    driving_video = "vid_in/driving.avi"
    fps = cap.get(cv2.CAP_PROP_FPS)
    # print(fps)

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    writer = cv2.VideoWriter(
        f"{root_path}/{driving_video}", fourcc, fps, (size_x, size_y))

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (size_x, size_y))
            # cv2.imshow("img", source_image)
            # cv2.imshow("vid", frame)
            writer.write(frame)

            if cv2.waitKey(ceil(fps)) == ord("q"):
                break
        else:
            break
    
    cv2.destroyAllWindows()
    cap.release()
    writer.release()
    return (source_image, driving_video)


def display_plot(source, driving_video, generated=None):
    titles = ["source_image", "driving_video"]

    driving = cv2.VideoCapture(f"{root_path}/{driving_video}")
    video_length = driving.get(cv2.CAP_PROP_FRAME_COUNT)
    source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)

    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(int(video_length)):
        cols = [source]
        cols.append(cv2.cvtColor(driving.read()[1], cv2.COLOR_BGR2RGB))
        if generated is not None:
            cols.append(cv2.cvtColor(generated[1], cv2.COLOR_BGR2RGB))
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)

    plt.close()
    return ani

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


**Displaying source image and driving video**

In [22]:
from IPython.display import HTML

image = cv2.imread("/content/gdrive/MyDrive/ColabNotebooks/ML/first-order-motion-model/data/Trump.png")
cap = cv2.VideoCapture("/content/gdrive/MyDrive/ColabNotebooks/ML/first-order-motion-model/data/vid_in/test_faked.mp4")
x, y = 256, 256                         # resizing the image and video

source_image, driving_video = img_vid_resize(image, cap, x, y)
print("done resize")

HTML(display_plot(source_image, driving_video).to_html5_video())

done resize


**Create a model and load checkpoints**

In [7]:
pwd

'/content/first-order-model'

In [10]:
from IPython.display import HTML
from demo import load_checkpoints

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/MyDrive/ColabNotebooks/ML/first-order-motion-model/data/vox-cpk.pth.tar')

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


**Perform image animation**

In [27]:
from demo import make_animation
from IPython.display import HTML

cap = cv2.VideoCapture(f"{root_path}/{driving_video}")
driving = []
while cap.isOpened():
  ret, frame = cap.read()
  if ret:
    driving.append(frame)
  else:
    break

predictions = make_animation(source_image, driving, generator, kp_detector, relative=True)
# predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)

# save resulting video
## imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder


HTML(display(source_image, driving, predictions).to_html5_video())

  0%|          | 0/444 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:4004: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 444/44